# Go Game Implementation 🎯

This notebook implements a functional Go game in Python, divided across multiple cells for clarity and modularity.

## Features ✨
- **9x9 Go board** (easily configurable to 19x19)
- **Move validation** with suicide rule enforcement
- **Stone capture** mechanics
- **Interactive gameplay** with text-based interface
- **Demo mode** with preset moves

## How to Play 🎮
1. Run all cells in order
2. Use `play_go_interactive()` for interactive gameplay
3. Enter moves as "row col" (e.g., "3 4")
4. Type "pass" to pass your turn
5. Type "quit" to exit

## Game Rules Implemented 📋
- Players alternate turns (Black goes first)
- Stones are captured when they have no liberties
- Suicide moves are prevented (unless they capture opponent stones)
- Basic Ko rule is not implemented (could be added later)

## Cell Structure 🏗️
1. **Cell 1**: Imports and constants
2. **Cell 2**: Board class with display and group logic
3. **Cell 3**: Move validation and capturing mechanics  
4. **Cell 4**: Game interface and demo functionality
5. **Cell 5**: This documentation

Enjoy playing Go! 🎯


In [1]:
# Go Game Implementation - Cell 1: Imports and Constants
import numpy as np
from typing import Tuple, List, Optional, Set

# Board constants
BOARD_SIZE = 9  # Can be changed to 19 for full Go
EMPTY, BLACK, WHITE = 0, 1, 2
STONE_STR = {EMPTY: '.', BLACK: '●', WHITE: '○'}

print("✅ Imports and constants loaded!")


✅ Imports and constants loaded!


In [2]:
# Go Game Implementation - Cell 2: Board Class
class GoBoard:
    def __init__(self, size: int = BOARD_SIZE):
        self.size = size
        self.board = np.zeros((size, size), dtype=int)
        self.current_player = BLACK
        self.captured_stones = {BLACK: 0, WHITE: 0}
        
    def display(self):
        """Display the current board state"""
        print("   " + " ".join([f"{i:2d}" for i in range(self.size)]))
        for i in range(self.size):
            row_str = f"{i:2d} "
            for j in range(self.size):
                row_str += f" {STONE_STR[self.board[i][j]]}"
            print(row_str)
        print(f"\nCurrent player: {STONE_STR[self.current_player]}")
        print(f"Captured - Black: {self.captured_stones[BLACK]}, White: {self.captured_stones[WHITE]}")
        
    def get_neighbors(self, row: int, col: int) -> List[Tuple[int, int]]:
        """Get valid neighboring positions"""
        neighbors = []
        for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            nr, nc = row + dr, col + dc
            if 0 <= nr < self.size and 0 <= nc < self.size:
                neighbors.append((nr, nc))
        return neighbors
        
    def get_group(self, row: int, col: int) -> Set[Tuple[int, int]]:
        """Get all stones in the same group (connected stones of same color)"""
        if self.board[row][col] == EMPTY:
            return set()
            
        color = self.board[row][col]
        group = set()
        stack = [(row, col)]
        
        while stack:
            r, c = stack.pop()
            if (r, c) in group:
                continue
            group.add((r, c))
            
            for nr, nc in self.get_neighbors(r, c):
                if self.board[nr][nc] == color and (nr, nc) not in group:
                    stack.append((nr, nc))
        return group
        
    def has_liberties(self, group: Set[Tuple[int, int]]) -> bool:
        """Check if a group has any liberties (empty adjacent spaces)"""
        for row, col in group:
            for nr, nc in self.get_neighbors(row, col):
                if self.board[nr][nc] == EMPTY:
                    return True
        return False

# Test the board
board = GoBoard()
board.display()
print("✅ Board class created!")


    0  1  2  3  4  5  6  7  8
 0  . . . . . . . . .
 1  . . . . . . . . .
 2  . . . . . . . . .
 3  . . . . . . . . .
 4  . . . . . . . . .
 5  . . . . . . . . .
 6  . . . . . . . . .
 7  . . . . . . . . .
 8  . . . . . . . . .

Current player: ●
Captured - Black: 0, White: 0
✅ Board class created!


In [3]:
# Go Game Implementation - Cell 3: Move Validation and Capturing Logic
class GoBoard(GoBoard):  # Extending the previous class
    def is_valid_move(self, row: int, col: int, color: int) -> bool:
        """Check if a move is valid"""
        # Check if position is on board and empty
        if not (0 <= row < self.size and 0 <= col < self.size):
            return False
        if self.board[row][col] != EMPTY:
            return False
        
        # Temporarily place the stone to test validity
        self.board[row][col] = color
        
        # Check if this move captures opponent stones
        opponent = WHITE if color == BLACK else BLACK
        captured_groups = []
        
        for nr, nc in self.get_neighbors(row, col):
            if self.board[nr][nc] == opponent:
                group = self.get_group(nr, nc)
                if not self.has_liberties(group):
                    captured_groups.extend(group)
        
        # Check if the placed stone has liberties (suicide rule)
        own_group = self.get_group(row, col)
        has_own_liberties = self.has_liberties(own_group)
        
        # Remove the temporarily placed stone
        self.board[row][col] = EMPTY
        
        # Move is valid if it either captures opponent stones or has liberties
        return len(captured_groups) > 0 or has_own_liberties
    
    def capture_stones(self, row: int, col: int, color: int) -> int:
        """Capture opponent stones after placing a stone"""
        opponent = WHITE if color == BLACK else BLACK
        captured_count = 0
        
        for nr, nc in self.get_neighbors(row, col):
            if self.board[nr][nc] == opponent:
                group = self.get_group(nr, nc)
                if not self.has_liberties(group):
                    # Remove captured stones
                    for gr, gc in group:
                        self.board[gr][gc] = EMPTY
                        captured_count += 1
        
        return captured_count
    
    def make_move(self, row: int, col: int) -> bool:
        """Make a move for the current player"""
        if not self.is_valid_move(row, col, self.current_player):
            print(f"Invalid move at ({row}, {col})")
            return False
        
        # Place the stone
        self.board[row][col] = self.current_player
        
        # Capture opponent stones
        captured = self.capture_stones(row, col, self.current_player)
        self.captured_stones[self.current_player] += captured
        
        # Switch players
        self.current_player = WHITE if self.current_player == BLACK else BLACK
        
        return True

print("✅ Move validation and capturing logic added!")


✅ Move validation and capturing logic added!


In [4]:
# Go Game Implementation - Cell 4: Game Interface and Demo
def play_go_interactive():
    """Interactive Go game"""
    game = GoBoard()
    print("🎮 Welcome to Go! Enter moves as 'row col' (e.g., '3 4') or 'quit' to exit.")
    print("Black (●) plays first!\n")
    
    game.display()
    
    while True:
        try:
            user_input = input(f"\n{STONE_STR[game.current_player]} player's turn: ").strip()
            
            if user_input.lower() == 'quit':
                print("Thanks for playing Go! 👋")
                break
            
            if user_input.lower() == 'pass':
                print(f"{STONE_STR[game.current_player]} passes their turn.")
                game.current_player = WHITE if game.current_player == BLACK else BLACK
                game.display()
                continue
                
            parts = user_input.split()
            if len(parts) != 2:
                print("Please enter row and column numbers separated by space (e.g., '3 4')")
                continue
                
            row, col = int(parts[0]), int(parts[1])
            
            if game.make_move(row, col):
                print(f"✅ Move placed at ({row}, {col})")
                game.display()
            else:
                print("❌ Invalid move! Try again.")
                
        except (ValueError, IndexError):
            print("Please enter valid numbers for row and column!")
        except KeyboardInterrupt:
            print("\nGame interrupted. Thanks for playing! 👋")
            break

# Demo function with preset moves
def demo_go_game():
    """Demonstrate Go gameplay with preset moves"""
    print("🎯 Go Game Demo - Watch some moves being played!")
    game = GoBoard()
    
    # Some demo moves
    demo_moves = [
        (2, 2, "Black opens in the center area"),
        (2, 6, "White responds on the opposite side"),
        (6, 2, "Black takes a corner approach"),
        (6, 6, "White mirrors the move"),
        (4, 4, "Black takes the center"),
        (3, 4, "White plays adjacent to black"),
        (4, 3, "Black extends"),
        (5, 4, "White continues to surround")
    ]
    
    game.display()
    
    for i, (row, col, description) in enumerate(demo_moves):
        print(f"\nMove {i+1}: {description}")
        if game.make_move(row, col):
            game.display()
        else:
            print(f"Move {i+1} was invalid!")
    
    print("🎬 Demo complete!")

# Run the demo
demo_go_game()
print("\n" + "="*50)
print("✅ Go game implementation complete!")
print("🎮 Run 'play_go_interactive()' to play interactively!")


🎯 Go Game Demo - Watch some moves being played!
    0  1  2  3  4  5  6  7  8
 0  . . . . . . . . .
 1  . . . . . . . . .
 2  . . . . . . . . .
 3  . . . . . . . . .
 4  . . . . . . . . .
 5  . . . . . . . . .
 6  . . . . . . . . .
 7  . . . . . . . . .
 8  . . . . . . . . .

Current player: ●
Captured - Black: 0, White: 0

Move 1: Black opens in the center area
    0  1  2  3  4  5  6  7  8
 0  . . . . . . . . .
 1  . . . . . . . . .
 2  . . ● . . . . . .
 3  . . . . . . . . .
 4  . . . . . . . . .
 5  . . . . . . . . .
 6  . . . . . . . . .
 7  . . . . . . . . .
 8  . . . . . . . . .

Current player: ○
Captured - Black: 0, White: 0

Move 2: White responds on the opposite side
    0  1  2  3  4  5  6  7  8
 0  . . . . . . . . .
 1  . . . . . . . . .
 2  . . ● . . . ○ . .
 3  . . . . . . . . .
 4  . . . . . . . . .
 5  . . . . . . . . .
 6  . . . . . . . . .
 7  . . . . . . . . .
 8  . . . . . . . . .

Current player: ●
Captured - Black: 0, White: 0

Move 3: Black takes a corner approa